# Setup

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch_geometric import seed_everything
from sklearn.preprocessing import MinMaxScaler,StandardScaler

import sys
sys.path.append("../../..")
from src.models import training_utils

seed = 4
seed_everything(seed)

data_folder = f"../../../data/processed/graph_data_nohubs/merged_types/split_dataset/seed_{seed}/"
feature_folder = "../../../data/processed/feature_data/"

# Load data

In [2]:
gtex_41 = pd.read_csv(feature_folder+"gtex.embedding.d41.tsv", sep="\t")
gtex_41 = gtex_41[~gtex_41.index.duplicated(keep='first')].reset_index(names="node_id")
node_data = pd.read_csv(data_folder+"tensor_df.csv",index_col=0)

In [67]:
node_values = node_data.node_type.value_counts()
node_values

gene_protein    17743
disease         16079
pathway          2017
Name: node_type, dtype: int64

In [4]:
node_data

,node_index,node_id,node_name,node_type,node_source,tensor_index,comunidades_infomap,comunidades_louvain,degree_gda,degree_pp,degree_dd,total_degree
0,0,1,A1BG,gene_protein,disgenet,0,NaN,NaN,2.0,12.0,0.0,14.0
1,653,10422,UBAC1,gene_protein,hippie,1,NaN,NaN,0.0,36.0,0.0,36.0
2,13168,63891,RNF123,gene_protein,disgenet,2,NaN,NaN,2.0,24.0,0.0,26.0
3,6638,284403,WDR62,gene_protein,disgenet,3,NaN,NaN,14.0,18.0,0.0,32.0
4,6611,284161,GDPD1,gene_protein,hippie,4,NaN,NaN,0.0,29.0,0.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...
35834,35265,R-HSA-5579013,Defective CYP7B1 causes SPG5A and CBAS3,pathway,primekg_REACTOME,2012,NaN,NaN,0.0,1.0,0.0,1.0
35835,35260,R-HSA-5579007,Defective ACY1 causes encephalopathy,pathway,primekg_REACTOME,2013,NaN,NaN,0.0,1.0,0.0,1.0
35836,35152,R-HSA-4687000,Defective MPDU1 causes MPDU1-CDG (CDG-1f),pathway,primekg_REACTOME,2014,NaN,NaN,0.0,1.0,0.0,1.0
35837,34551,R-HSA-1855231,Synthesis of IPs in the ER lumen,pathway,primekg_REACTOME,2015,NaN,NaN,0.0,1.0,0.0,1.0


In [5]:
gtex_41

,node_id,Adipose - Subcutaneous,Adipose - Visceral (Omentum),Adrenal Gland,Artery - Aorta,Artery - Coronary,Artery - Tibial,Brain - Caudate (basal ganglia),Brain - Cerebellar Hemisphere,Brain - Cerebellum,...,Pituitary,Prostate,Skin - Not Sun Exposed (Suprapubic),Skin - Sun Exposed (Lower leg),Spleen,Stomach,Testis,Thyroid,Vagina,Whole Blood
0,100420257,0.055851,0.192778,0.081072,0.082686,0.076427,0.033635,0.088682,0.085177,0.106887,...,0.268007,0.169910,0.103337,0.081238,1.691810,0.137969,0.637723,0.151513,0.325006,10.925670
1,728481,8.605529,6.632387,5.824766,5.009951,5.338176,4.829056,6.631166,9.762946,9.314376,...,5.349413,4.693062,4.318812,5.266853,6.719691,2.643370,5.734409,3.529109,3.675271,24.397447
2,101928626,0.251195,0.246317,0.176912,0.234222,0.246855,0.281459,0.219588,0.298781,0.362751,...,0.384611,0.247489,0.251922,0.245274,0.583390,0.216590,1.502332,0.259857,0.264840,1.955185
3,100887749,434.570831,218.656711,393.786646,47.866333,90.972202,263.038339,2172.785775,597.699926,586.966950,...,32.761514,130.306713,193.886822,495.856298,130.911066,255.913345,110.865193,347.858681,85.560646,235.904951
4,100652939,2270.118713,1498.787473,1530.811037,364.155415,593.903538,1253.640854,11439.218597,2717.040622,2628.444677,...,585.562758,936.151586,853.588200,1921.469693,482.662748,1560.786691,519.118761,1970.212040,437.978451,762.195611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11997,57706,30.708041,26.280459,29.129426,24.377011,24.952587,33.450177,34.635990,30.939456,34.198858,...,27.310242,21.331650,20.229137,23.296021,32.403528,23.680096,36.082690,27.092825,21.835841,108.406217
11998,9355,0.319327,0.190671,0.095634,0.189582,0.442795,0.085160,159.049640,0.405770,1.966181,...,13.792214,0.103582,16.797929,8.995891,0.288182,0.055054,7.838468,0.080973,0.189550,0.183251
11999,100505588,0.005494,0.001692,0.002733,0.005181,0.007445,0.001869,1.998111,0.024648,0.114554,...,1.404057,0.005221,0.241453,0.136293,0.009131,0.001954,0.706724,0.004519,0.003776,0.005522
12000,10783,68.375814,65.622502,27.786967,66.398059,66.950275,81.213605,147.409926,20.023448,28.417042,...,19.922139,70.508481,22.497264,24.484488,50.291844,64.118642,24.452221,31.003894,30.389123,92.989325


In [3]:
#esto es xq están los complejos, que tienen ids raros y cuando mergeo hace mal las cosas sino
genes = node_data[node_data.node_id.apply(lambda x: x.isnumeric())]
id_genes = genes.node_id.astype(int).values
index_genes = genes.node_index.values
num_features = len(set(id_genes) & set(gtex_41.node_id.values))
print(f"Número de genes con feature GTEx: {num_features} \nCobertura de features: {round((num_features*100)/len(genes),2)}% \nMissing feature rate: {round(((len(genes) - num_features)*100)/len(genes),2)}%")

# genes_features = genes[["node_index","node_id"]].merge(gtex_41, left_on = "node_id", right_index=True).set_index("node_index").drop(columns="node_id")

Número de genes con feature GTEx: 7451 
Cobertura de features: 43.01% 
Missing feature rate: 56.99%


In [73]:
genes_features = pd.merge(gtex_41,genes[["node_id","node_index"]].astype(int),left_on="node_id", right_on="node_id").set_index("node_index").drop(columns="node_id")
genes_features

,Adipose - Subcutaneous,Adipose - Visceral (Omentum),Adrenal Gland,Artery - Aorta,Artery - Coronary,Artery - Tibial,Brain - Caudate (basal ganglia),Brain - Cerebellar Hemisphere,Brain - Cerebellum,Brain - Cortex,...,Pituitary,Prostate,Skin - Not Sun Exposed (Suprapubic),Skin - Sun Exposed (Lower leg),Spleen,Stomach,Testis,Thyroid,Vagina,Whole Blood
node_index,,,,,,,,,,,,,,,,,,,,,
929,670.339699,143.474417,214.738274,42.272098,85.610230,579.046751,4573.460742,1416.519387,1182.906108,2382.117662,...,76.682057,153.698475,133.536976,977.324970,98.955411,203.115354,56.596059,1170.710169,59.322659,629.146347
3057,1.021395,5.888943,9.803881,10.764966,6.578287,14.829003,5.709347,0.503772,0.787666,1.664383,...,50.301461,29.632569,1.044601,1.314332,26.830951,8.080823,8.107697,7.206900,13.455339,0.089460
5935,120.829005,123.495634,111.707228,119.706769,115.984956,134.974131,89.887375,153.262994,151.959356,129.553738,...,116.345698,126.419531,207.444380,220.891927,129.912224,133.374089,162.522372,116.353100,144.336414,119.719879
7551,22.075988,24.393677,24.761539,30.602361,30.957489,23.253785,18.889650,54.153405,63.542813,37.756098,...,46.690524,37.872641,41.862121,40.076916,50.495530,33.636373,53.234991,27.727856,40.034597,30.122658
16154,1.281198,1.591965,6.960659,1.774428,2.022627,1.605353,0.355029,0.250737,0.326711,0.482001,...,1.509277,15.293920,175.193403,165.865747,5.195992,5.626856,2.480773,2.848963,101.578783,3.880818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11362,16.952397,20.929488,29.844665,14.466432,17.125201,16.004584,69.351409,74.768002,62.575079,97.935852,...,103.766584,38.617818,33.986961,30.146878,56.947833,53.662612,188.668667,75.365028,46.150856,25.217527
6745,0.260612,15.158173,0.254693,0.503879,0.636808,0.237142,73.369808,12.932179,27.794828,69.994610,...,15.622855,0.555831,0.674778,0.662729,4.612809,0.450001,5.947310,1.204611,0.188299,1.338138
12492,30.708041,26.280459,29.129426,24.377011,24.952587,33.450177,34.635990,30.939456,34.198858,57.612621,...,27.310242,21.331650,20.229137,23.296021,32.403528,23.680096,36.082690,27.092825,21.835841,108.406217


Hay que normalizar esto

In [48]:
def fill_missing_with_mean(feature_matrix,node_type,node_data):
    global_mean = feature_matrix.mean(axis=0).values
    # feature_matrix.index = feature_matrix.index.astype(int)
    has_feature = feature_matrix.index.values.astype(int)
    all_index = node_data[node_data.node_type == node_type].node_index.values
    no_feature = list(set(all_index) - set(has_feature))

    global_mean_matrix = np.tile(global_mean, (len(no_feature), 1))
    global_mean_df = pd.DataFrame(global_mean_matrix, index=no_feature, columns=feature_matrix.columns)
    full_feature_df = (
        pd.concat([feature_matrix, global_mean_df])
        .reset_index()
        .rename(columns={"index": "node_index"})
    )

    full_feature_df = (
        pd.merge(
            full_feature_df,
            node_data[["node_index", "tensor_index"]],
            left_on="node_index",
            right_on="node_index",
        )
        .sort_values(by="tensor_index")
        .drop(columns=["node_index"])
        .set_index("tensor_index")
    )

    return full_feature_df

In [6]:
genes_features

,Adipose - Subcutaneous,Adipose - Visceral (Omentum),Adrenal Gland,Artery - Aorta,Artery - Coronary,Artery - Tibial,Brain - Caudate (basal ganglia),Brain - Cerebellar Hemisphere,Brain - Cerebellum,Brain - Cortex,...,Pituitary,Prostate,Skin - Not Sun Exposed (Suprapubic),Skin - Sun Exposed (Lower leg),Spleen,Stomach,Testis,Thyroid,Vagina,Whole Blood
node_index,,,,,,,,,,,,,,,,,,,,,
13168,60.701321,60.372018,89.558808,67.531346,65.640758,76.084979,60.665246,102.655701,118.754379,106.547014,...,71.969190,63.798157,65.254730,68.326037,72.949693,82.439418,91.521777,70.389018,59.003752,210.376726
11965,148.103712,135.544269,99.128136,131.174530,128.815065,177.493447,121.643891,132.645841,107.342672,146.988699,...,98.586530,115.750700,132.657088,134.812608,104.300728,132.424499,174.692476,107.613460,149.980862,302.824545
569,0.856786,0.634610,0.213634,0.472465,0.241922,0.375022,0.326830,0.192518,0.407356,0.279494,...,0.442020,5.112073,11.459986,13.626242,7.156095,0.417004,0.624152,0.432827,522.411363,180.027285
18356,139.859074,98.954883,253.510921,222.477134,235.394031,165.406048,76.124915,122.257870,126.928312,80.217776,...,164.162153,160.841348,116.358505,129.275484,250.897857,154.341717,103.236062,143.441688,106.350118,127.672174
5099,574.243653,485.222640,633.424315,761.489288,614.827116,690.925737,393.389690,491.657180,574.369251,516.199121,...,638.074190,650.195941,918.226668,1010.859465,492.799791,740.612315,953.375021,562.756987,535.413654,737.079307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18316,93.735696,119.209783,87.026852,275.147225,241.005284,121.595423,212.452412,762.516077,857.576342,241.507962,...,118.311130,303.315259,40.009677,38.601198,168.462566,125.564566,37.507900,99.298265,172.773899,311.757145
18353,38.233477,33.549466,76.276860,7.172284,14.237015,20.843591,315.083627,307.416770,256.370193,447.740657,...,44.290121,35.606706,17.791488,20.991718,89.573309,23.900320,14.226956,48.982244,14.180444,63.199478
18373,8.952212,14.349148,5.074201,2.829493,4.031892,2.165072,25.229236,0.721265,0.927975,5.775865,...,109.817849,12.507884,2.969153,2.964398,6.015801,3.718942,146.055083,13.252594,2.460000,15.679389


In [76]:
aver = fill_missing_with_mean(genes_features,"gene_protein",node_data)
aver

,Adipose - Subcutaneous,Adipose - Visceral (Omentum),Adrenal Gland,Artery - Aorta,Artery - Coronary,Artery - Tibial,Brain - Caudate (basal ganglia),Brain - Cerebellar Hemisphere,Brain - Cerebellum,Brain - Cortex,...,Pituitary,Prostate,Skin - Not Sun Exposed (Suprapubic),Skin - Sun Exposed (Lower leg),Spleen,Stomach,Testis,Thyroid,Vagina,Whole Blood
tensor_index,,,,,,,,,,,,,,,,,,,,,
0,114.105049,117.410596,108.207289,136.663747,124.855427,138.566357,97.944332,94.257898,89.909390,101.583302,...,98.468077,91.307941,108.400027,109.303225,93.503711,102.979477,69.380585,99.124605,109.644473,204.492098
1,114.105049,117.410596,108.207289,136.663747,124.855427,138.566357,97.944332,94.257898,89.909390,101.583302,...,98.468077,91.307941,108.400027,109.303225,93.503711,102.979477,69.380585,99.124605,109.644473,204.492098
2,60.701321,60.372018,89.558808,67.531346,65.640758,76.084979,60.665246,102.655701,118.754379,106.547014,...,71.969190,63.798157,65.254730,68.326037,72.949693,82.439418,91.521777,70.389018,59.003752,210.376726
3,114.105049,117.410596,108.207289,136.663747,124.855427,138.566357,97.944332,94.257898,89.909390,101.583302,...,98.468077,91.307941,108.400027,109.303225,93.503711,102.979477,69.380585,99.124605,109.644473,204.492098
4,114.105049,117.410596,108.207289,136.663747,124.855427,138.566357,97.944332,94.257898,89.909390,101.583302,...,98.468077,91.307941,108.400027,109.303225,93.503711,102.979477,69.380585,99.124605,109.644473,204.492098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17738,114.105049,117.410596,108.207289,136.663747,124.855427,138.566357,97.944332,94.257898,89.909390,101.583302,...,98.468077,91.307941,108.400027,109.303225,93.503711,102.979477,69.380585,99.124605,109.644473,204.492098
17739,114.105049,117.410596,108.207289,136.663747,124.855427,138.566357,97.944332,94.257898,89.909390,101.583302,...,98.468077,91.307941,108.400027,109.303225,93.503711,102.979477,69.380585,99.124605,109.644473,204.492098
17740,7.835221,19.296186,13.672853,7.060225,11.677229,4.506695,8.158906,154.879151,130.232258,17.361173,...,35.192444,15.793536,4.060467,3.968935,16.269305,21.164795,10.593514,12.422327,18.881452,3.060132


In [77]:
scaler = MinMaxScaler([-1,1])
scaler.fit(genes_features.values)
scaled_values = scaler.transform(genes_features.values)
scaled_df = pd.DataFrame(scaled_values, index=genes_features.index, columns=genes_features.columns)
scaled_df

,Adipose - Subcutaneous,Adipose - Visceral (Omentum),Adrenal Gland,Artery - Aorta,Artery - Coronary,Artery - Tibial,Brain - Caudate (basal ganglia),Brain - Cerebellar Hemisphere,Brain - Cerebellum,Brain - Cortex,...,Pituitary,Prostate,Skin - Not Sun Exposed (Suprapubic),Skin - Sun Exposed (Lower leg),Spleen,Stomach,Testis,Thyroid,Vagina,Whole Blood
node_index,,,,,,,,,,,,,,,,,,,,,
929,-0.880991,-0.978194,-0.982054,-0.997303,-0.993280,-0.955779,-0.121666,-0.250200,-0.521685,-0.493282,...,-0.998375,-0.963536,-0.976666,-0.845157,-0.980149,-0.961910,-0.971261,-0.958041,-0.990792,-0.979311
3057,-0.999819,-0.999105,-0.999181,-0.999313,-0.999484,-0.998868,-0.998904,-0.999733,-0.999682,-0.999646,...,-0.998934,-0.992970,-0.999817,-0.999792,-0.994618,-0.998485,-0.995883,-0.999742,-0.997912,-0.999997
5935,-0.978548,-0.981230,-0.990664,-0.992361,-0.990896,-0.989692,-0.982737,-0.918874,-0.938554,-0.972442,...,-0.997535,-0.970007,-0.963752,-0.965003,-0.973939,-0.974988,-0.917473,-0.995830,-0.977597,-0.996063
7551,-0.996081,-0.996293,-0.997931,-0.998047,-0.997570,-0.998224,-0.996372,-0.971335,-0.974306,-0.991969,...,-0.999011,-0.991015,-0.992685,-0.993650,-0.989870,-0.993692,-0.972968,-0.999006,-0.993786,-0.999009
16154,-0.999773,-0.999758,-0.999418,-0.999887,-0.999841,-0.999877,-0.999932,-0.999867,-0.999868,-0.999897,...,-0.999968,-0.996372,-0.969387,-0.973721,-0.998958,-0.998945,-0.998740,-0.999898,-0.984233,-0.999872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11362,-0.996990,-0.996819,-0.997506,-0.999077,-0.998656,-0.998778,-0.986681,-0.960423,-0.974697,-0.979167,...,-0.997801,-0.990838,-0.994061,-0.995224,-0.988576,-0.989937,-0.904197,-0.997299,-0.992837,-0.999171
6745,-0.999954,-0.997696,-0.999979,-0.999968,-0.999950,-0.999982,-0.985909,-0.993155,-0.988761,-0.985111,...,-0.999669,-0.999868,-0.999882,-0.999895,-0.999075,-0.999916,-0.996980,-0.999957,-0.999971,-0.999956
12492,-0.994548,-0.996006,-0.997566,-0.998444,-0.998041,-0.997445,-0.993348,-0.983623,-0.986172,-0.987745,...,-0.999421,-0.994939,-0.996465,-0.996309,-0.993500,-0.995559,-0.981678,-0.999029,-0.996611,-0.996435


In [78]:
scaler = StandardScaler()
scaler.fit(genes_features.values)
scaled_values = scaler.transform(genes_features.values)
scaled_df = pd.DataFrame(scaled_values, index=genes_features.index, columns=genes_features.columns)
scaled_df

,Adipose - Subcutaneous,Adipose - Visceral (Omentum),Adrenal Gland,Artery - Aorta,Artery - Coronary,Artery - Tibial,Brain - Caudate (basal ganglia),Brain - Cerebellar Hemisphere,Brain - Cerebellum,Brain - Cortex,...,Pituitary,Prostate,Skin - Not Sun Exposed (Suprapubic),Skin - Sun Exposed (Lower leg),Spleen,Stomach,Testis,Thyroid,Vagina,Whole Blood
node_index,,,,,,,,,,,,,,,,,,,,,
929,1.569984,0.072055,0.258507,-0.151996,-0.076242,0.727010,16.674952,6.521424,5.694485,8.780109,...,-0.016721,0.251447,0.076916,2.540039,0.020987,0.329989,-0.100094,1.546556,-0.142304,0.359949
3057,-0.319181,-0.308308,-0.238784,-0.202731,-0.229777,-0.204228,-0.343651,-0.462398,-0.464322,-0.384690,...,-0.036969,-0.248564,-0.328495,-0.316001,-0.256661,-0.312730,-0.479725,-0.132659,-0.272011,-0.173258
5935,0.018979,0.016822,0.008493,-0.027305,-0.017233,-0.005929,-0.030019,0.291014,0.323279,0.107687,...,0.013722,0.141507,0.303064,0.326535,0.140157,0.100163,0.729237,0.024865,0.098105,-0.071855
7551,-0.259754,-0.257150,-0.202488,-0.170788,-0.182415,-0.190323,-0.294543,-0.197796,-0.137369,-0.245736,...,-0.039741,-0.215355,-0.203598,-0.202573,-0.165563,-0.228514,-0.126409,-0.103043,-0.196848,-0.147801
16154,-0.318448,-0.320187,-0.245684,-0.217208,-0.238627,-0.226053,-0.363600,-0.463645,-0.466723,-0.389242,...,-0.074419,-0.306352,0.204380,0.165515,-0.339947,-0.320817,-0.523780,-0.138949,-0.022809,-0.170044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11362,-0.274215,-0.266727,-0.190154,-0.196771,-0.209287,-0.202287,-0.106532,-0.096125,-0.142411,-0.014043,...,0.004067,-0.212352,-0.227695,-0.231630,-0.140724,-0.162519,0.933944,-0.034291,-0.179552,-0.151958
6745,-0.321328,-0.282682,-0.261956,-0.219254,-0.241319,-0.228312,-0.091560,-0.401100,-0.323615,-0.121617,...,-0.063586,-0.365750,-0.329626,-0.317908,-0.342192,-0.337877,-0.496639,-0.141322,-0.309529,-0.172199
12492,-0.235390,-0.251934,-0.191889,-0.180812,-0.194081,-0.173494,-0.235875,-0.312288,-0.290250,-0.169288,...,-0.054616,-0.282019,-0.269792,-0.251678,-0.235209,-0.261324,-0.260700,-0.103959,-0.248312,-0.081445
